# 🌧️ Timer-XL Peru Rainfall Prediction - Google Colab

This notebook demonstrates the complete pipeline for training Timer-XL on Peru rainfall data.

**Steps:**
1. Setup environment
2. Upload ERA5 data
3. Preprocess data
4. Train Timer-XL with transfer learning
5. Evaluate results

## 1. Setup Environment

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Clone repository
!git clone https://github.com/ChristianPE1/test-openltm-code.git
%cd test-openltm-code

In [ ]:
# Install dependencies
!pip install -r requirements.txt

In [ ]:
# Mount Google Drive (to download checkpoint.pth and save training results)
from google.colab import drive
drive.mount('/content/drive')

# Create directories in Drive
!mkdir -p '/content/drive/MyDrive/timer_xl_peru/'
!mkdir -p '/content/drive/MyDrive/timer_xl_peru/checkpoints/'
!mkdir -p '/content/drive/MyDrive/timer_xl_peru/results/'

print("✅ Google Drive mounted")
print("📁 Created folders:")
print("   - /content/drive/MyDrive/timer_xl_peru/checkpoints/  (for trained models)")
print("   - /content/drive/MyDrive/timer_xl_peru/results/      (for outputs)")

## 2. Verificar Datos ERA5

**Los archivos .nc ya están en el repositorio** (datasets/raw_era5/)  
Solo necesitas verificar que se clonaron correctamente.

In [ ]:
# Verify ERA5 files are in the repository
!ls -lh datasets/raw_era5/

print("\n✅ Expected files:")
print("   - era5_peru_2022.nc")
print("   - era5_peru_2023.nc")
print("   - era5_peru_2024.nc")

## 3. Preprocess Data

In [ ]:
# Run preprocessing script
!python preprocessing/preprocess_era5_peru.py \
    --input_dir datasets/raw_era5 \
    --output_dir datasets/processed \
    --years 2022,2023,2024 \
    --target_horizon 24 \
    --threshold 0.1

print("\n✅ Preprocessing complete!")
print("📊 Output files saved to: datasets/processed/")

In [ ]:
# Load processed data for quick inspection
import pandas as pd
import json

df = pd.read_csv('datasets/processed/peru_rainfall.csv')
print(f"Dataset shape: {df.shape}")
print(f"\nFirst few rows:")
print(df.head())

# Load statistics
with open('datasets/processed/preprocessing_stats.json') as f:
    stats = json.load(f)
print(f"\nStatistics:")
print(json.dumps(stats, indent=2))

## 4. Train Timer-XL

In [ ]:
# Copy pre-trained checkpoint from Google Drive
import os

checkpoint_path = 'checkpoints/timer_xl/checkpoint.pth'

if not os.path.exists(checkpoint_path):
    print("📥 Copying pre-trained Timer-XL checkpoint from Google Drive...")
    os.makedirs('checkpoints/timer_xl', exist_ok=True)
    
    # Copy from Drive
    !cp '/content/drive/MyDrive/timer_xl_peru/checkpoints/checkpoint.pth' checkpoints/timer_xl/
    
    if os.path.exists(checkpoint_path):
        print("✅ Checkpoint copied successfully!")
    else:
        print("❌ ERROR: Checkpoint not found in Drive")
        print("📋 Please upload checkpoint.pth to:")
        print("   /content/drive/MyDrive/timer_xl_peru/checkpoints/checkpoint.pth")
        print("\n   Download from: https://cloud.tsinghua.edu.cn/f/01c35ca13f474176be7b/")
else:
    print(f"✅ Checkpoint already exists: {checkpoint_path}")

In [ ]:
# Train model
# Note: Adjust batch_size if you get OOM errors

!python run.py \
  --task_name forecast \
  --is_training 1 \
  --root_path datasets/processed/ \
  --data_path peru_rainfall.csv \
  --model_id peru_rainfall_transfer_learning \
  --model timer_xl_classifier \
  --data PeruRainfall \
  --seq_len 1440 \
  --input_token_len 96 \
  --output_token_len 96 \
  --test_seq_len 1440 \
  --test_pred_len 2 \
  --e_layers 8 \
  --d_model 1024 \
  --d_ff 2048 \
  --n_heads 8 \
  --dropout 0.1 \
  --activation relu \
  --batch_size 128 \
  --learning_rate 1e-5 \
  --train_epochs 50 \
  --patience 10 \
  --gpu 0 \
  --cosine \
  --tmax 50 \
  --use_norm \
  --adaptation \
  --pretrain_model_path checkpoints/timer_xl/checkpoint.pth \
  --loss CE \
  --checkpoints results/peru_rainfall/ \
  --use_focal_loss \
  --itr 1

## 5. Save Checkpoint to Drive

Prevent losing your trained model!

In [ ]:
# Copy best checkpoint to Google Drive (prevent losing trained model!)
import shutil
import os

# Find the latest checkpoint
results_path = 'results/peru_rainfall/peru_rainfall_transfer_learning/'

if os.path.exists(results_path):
    # Copy entire results folder to Drive
    drive_results = '/content/drive/MyDrive/timer_xl_peru/results/peru_rainfall_transfer_learning/'
    os.makedirs(drive_results, exist_ok=True)
    
    print("💾 Copying results to Google Drive...")
    !cp -r {results_path}* {drive_results}
    
    print("✅ Checkpoint and results saved to Google Drive!")
    print(f"📁 Location: {drive_results}")
else:
    print("⚠️ No results found yet. Training may still be in progress.")

## 6. Quick Evaluation

In [ ]:
# Load test results (if available)
import os
import json

results_dir = 'results/peru_rainfall/peru_rainfall_transfer_learning/'

# Find metrics file
for file in os.listdir(results_dir):
    if file.startswith('test_metrics') and file.endswith('.json'):
        with open(os.path.join(results_dir, file)) as f:
            metrics = json.load(f)
        print("Test Metrics:")
        print(json.dumps(metrics, indent=2))

## 🎉 Training Complete!

**Next steps:**
1. Download results from `results/peru_rainfall/`
2. Analyze confusion matrix and classification report
3. Try different context lengths (seq_len)
4. Experiment with different hyperparameters